In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 一、预处理

导入相关函数

In [2]:
import time

# We have three versions of our implementations
# Version1: without using socket and no DP+PixelDP
# Version2: with using socket but no DP+PixelDP
# Version3: without using socket but with DP+PixelDP

# This program is Version1: Single program simulation
# ============================================================================
import torch
from torch import nn
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import math
import os.path
import pandas as pd
from sklearn.model_selection import train_test_split
from PIL import Image
from glob import glob
from pandas import DataFrame
from collections import OrderedDict


import torchvision.datasets as datasets
import random
import numpy as np
import os
from torchvision.datasets import ImageFolder

import matplotlib

pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import copy

SEED = 1234

def init_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    if torch.cuda.is_available():
        torch.backends.cudnn.deterministic = True
        print(torch.cuda.get_device_name(0))
init_seeds(SEED)

NVIDIA GeForce GTX 1660 Ti


## 1.1模型初始化

In [3]:
from torch import nn
import torch.nn.functional as F

def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)

class ResNet18_client_side(nn.Module):
    def __init__(self, block, num_blocks):
        super(ResNet18_client_side, self).__init__()
        self.in_planes = 64
        self.conv1 = conv3x3(3, 64)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        return out


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(in_planes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet18_server_side(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10, pool_size=4):  # Add a new argument pool_size
        super(ResNet18_server_side, self).__init__()
        self.in_planes = 64
        self.pool_size = pool_size  # Add this line to store pool_size
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.layer2(x)
        out = self.layer3(out)
        out = self.layer4(out)
        # print("Output shape before pooling:", out.shape)
        out = F.avg_pool2d(out, kernel_size=self.pool_size)  # Use self.pool_size instead of 8
        # print("Output shape after pooling:", out.shape)
        out = out.view(out.size(0), -1)
        y_hat = self.linear(out)
        return y_hat

In [4]:
import logging

def init_logging(program_name):
    logging.basicConfig(filename=f'{program_name}.log', level=logging.INFO)
    logging.getLogger().addHandler(logging.StreamHandler())
    
# ===================================================================
program = "SFLV1_label_random_"
print(f"---------{program}----------")  # this is to identify the program in the slurm outputs files
init_logging(program)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ===================================================================

# No. of users
num_users = 20
epochs = 100
frac = 1  # participation of clients; if 1 then 100% clients participate in SFLV1
lr = 0.0001
poisoned_frac = 0.1

# CIFAR10\HAM10000
dataset_choice = 'CIFAR10'

---------SFLV1_label_random_----------


In [5]:
def init_models(device, dataset_choice):
    net_glob_client = ResNet18_client_side(BasicBlock, [2, 2, 2, 2]).to(device)
    if torch.cuda.device_count() > 1:
        logging.info(f"We use {torch.cuda.device_count()} GPUs")
        net_glob_client = nn.DataParallel(net_glob_client)

    net_glob_client.to(device)
    logging.info(net_glob_client)

    if dataset_choice == 'HAM10000':
        num_classes = 7
        pool_size = 8
    elif dataset_choice == 'CIFAR10':
        num_classes = 10
        pool_size = 4
    else:
        raise ValueError('Invalid dataset choice.')

    net_glob_server = ResNet18_server_side(BasicBlock, [2,2,2,2], num_classes=num_classes,pool_size=pool_size)
    if torch.cuda.device_count() > 1:
        logging.info(f"We use {torch.cuda.device_count()} GPUs")
        net_glob_server = nn.DataParallel(net_glob_server)

    net_glob_server.to(device)
    logging.info(net_glob_server)

    return net_glob_client, net_glob_server

net_glob_client, net_glob_server = init_models(device, dataset_choice)

ResNet18_client_side(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentu

## 1.2 变量初始化

In [6]:
# ===================================================================================
# For Server Side Loss and Accuracy
loss_train_collect = []
acc_train_collect = []
loss_test_collect = []
acc_test_collect = []
batch_acc_train = []
batch_loss_train = []
batch_acc_test = []
batch_loss_test = []

criterion = nn.CrossEntropyLoss()
count1 = 0
count2 = 0

# ====================================================================================================
#                                  Server Side Program
# ====================================================================================================
# Federated averaging: FedAvg
# to print train - test together in each round-- these are made global
acc_avg_all_user_train = 0
loss_avg_all_user_train = 0
loss_train_collect_user = []
acc_train_collect_user = []
loss_test_collect_user = []
acc_test_collect_user = []

# （即权重和偏置）保存到w_glob_server中。
w_glob_server = net_glob_server.state_dict()
w_locals_server = []

# client idx collector
idx_collect = []  # 初始化一个空列表，用于收集选择的客户端的索引。
l_epoch_check = False  # 初始化一个布尔变量，用于指示是否进行了本地训练轮次的检查。
fed_check = False  # 初始化一个布尔变量，用于指示是否完成了联邦学习。
# Initialization of net_model_server and net_server (server-side model)
net_model_server = [net_glob_server for i in range(num_users)]  # 该列表包含了每个客户端的初始模型。
net_server = copy.deepcopy(net_model_server[0]).to(device)  # 初始化为net_model_server的第一个元素的深拷贝，并将其移到GPU上。


In [7]:
# To print in color -------test/train of the client side
def prRed(skk): print("\033[91m {}\033[00m" .format(skk))
def prGreen(skk): print("\033[92m {}\033[00m" .format(skk))

def calculate_accuracy(fx, y):
    preds = fx.max(1, keepdim=True)[1]
    correct = preds.eq(y.view_as(preds)).sum()
    acc = 100.00 *correct.float()/preds.shape[0]
    return acc

def subtract_weights(w1, w2):
    """
    计算 w1 和 w2 之间的差，即 w1 - w2。

    Args:
        w1 (OrderedDict): 权重字典 1
        w2 (OrderedDict): 权重字典 2

    Returns:
        OrderedDict: w1 和 w2 之间的权重差异
    """
    diff = OrderedDict()
    for key in w1.keys():
        diff[key] = w1[key] - w2[key]
    return diff


# Federated averaging: FedAvg
def FedAvg(w):
    w_avg = copy.deepcopy(w[0])
    for k in w_avg.keys():
        for i in range(1, len(w)):
            w_avg[k] += w[i][k]
        w_avg[k] = torch.div(w_avg[k], len(w))
    return w_avg

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

def remove_anomalies_kmeans(w_locals, w_glob, n_clusters, remove_ratio):
    gradients_diffs = [subtract_weights(w_local, w_glob) for w_local in w_locals]
    gradients_list = []

    for diff in gradients_diffs:
        diff_list = [v.cpu().numpy().flatten() for v in diff.values()]
        gradients_list.append(np.concatenate(diff_list))

    scaler = StandardScaler()
    scaled_gradients = scaler.fit_transform(gradients_list)

    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(scaled_gradients)
    cluster_labels = kmeans.labels_

    cluster_sizes = np.bincount(cluster_labels)
    smallest_cluster = np.argmin(cluster_sizes)

    remove_count = int(remove_ratio * len(w_locals))
    smallest_cluster_indices = np.argsort(np.linalg.norm(kmeans.transform(gradients_list), axis=1))[:remove_count]

    w_locals = [w for idx, w in enumerate(w_locals) if idx not in smallest_cluster_indices]

    return w_locals,smallest_cluster_indices

def krum_aggregation(weight_dicts, num_to_select):
    # ...（weights_to_array、array_to_weights和pairwise_distances函数保持不变）
    def weights_to_array(weight_dict):
        """
        将权重字典转换为 numpy 数组。
        :param weight_dict: 权重字典，其中每个值都是一个 torch.Tensor 张量。
        :return: 一个 numpy 数组，其中包含了所有权重张量的扁平化数组。
        """
        # 初始化 weight_list 列表，用于存储 weight_dict 中每个权重张量的扁平化数组。
        weight_list = []
        for key in weight_dict:
            # 将权重张量转换为 numpy 数组，并使用 flatten 函数将其扁平化
            weight_list.append(weight_dict[key].cpu().numpy().flatten())
        # 将 weight_list 中的数组连接成一个 numpy 数组，并返回该数组
        return np.concatenate(weight_list)

    def array_to_weights(array, weight_dict_template):
        """
        从 numpy 数组中提取权重张量，并将它们保存到一个新的权重字典中。
        :param array: 包含所有权重张量的扁平化 numpy 数组。
        :param weight_dict_template: 一个权重字典模板，其中包含了所有权重张量的形状。
        :return: 一个新的权重字典，其中包含了从数组中提取的权重张量。
        """
        # 初始化一个新的有序字典 new_weight_dict，用于存储从 numpy 数组中提取的权重张量
        new_weight_dict = OrderedDict()
        # 初始化一个索引变量 idx，用于跟踪从数组中提取权重的位置
        idx = 0
        # 遍历权重字典模板中的每个键 key
        for key in weight_dict_template:
            # 获取权重张量的大小 size
            size = weight_dict_template[key].numel()
            # 从 numpy 数组中提取一个与权重张量相同大小的一维切片，并使用 reshape 函数重新塑形
            # 将 numpy 数组转换为 torch.Tensor 张量，并将其存储到新的有序字典 new_weight_dict 中
            new_weight_dict[key] = torch.from_numpy(array[idx:idx + size].reshape(weight_dict_template[key].shape))
            # 更新索引变量 idx 的值，跳过已经提取的权重张量
            idx += size
        # 返回新的有序字典 new_weight_dict
        return new_weight_dict

    def pairwise_distances(weight_updates):
        # 获取客户端数量
        n_clients = len(weight_updates)
        # 初始化一个距离矩阵，矩阵大小为 n_clients * n_clients
        distances = np.zeros((n_clients, n_clients))

        # 遍历所有的客户端对
        for i in range(n_clients):
            for j in range(i + 1, n_clients):
                # 计算 i 和 j 客户端之间的欧氏距离
                dist = np.linalg.norm(weight_updates[i] - weight_updates[j])
                # 在距离矩阵中记录距离
                distances[i, j] = dist
                distances[j, i] = dist

        # 返回距离矩阵
        return distances

    weight_arrays = [weights_to_array(weight_dict) for weight_dict in weight_dicts]
    n_clients = len(weight_arrays)
    distances = pairwise_distances(weight_arrays)

    krum_scores = []
    for i in range(n_clients):
        sorted_distances = np.sort(distances[i])
        # 计算当前客户端的Krum分数，即距离最大的n_clients - num_to_select - 1个客户端之间的距离总和。
        krum_score = np.sum(sorted_distances[-(n_clients - num_to_select - 1):])
        krum_scores.append(krum_score)

    # 使用np.argpartition函数找到具有最低Krum分数的num_to_select个客户端的索引。
    best_clients_indices = np.argpartition(krum_scores, num_to_select)[:num_to_select]

    # 从权重数组中选择最佳客户端的权重。
    selected_weight_arrays = [weight_arrays[i] for i in best_clients_indices]
    # 计算选定客户端的权重数组的平均值，得到聚合后的权重数组。
    aggregated_weight_array = np.mean(selected_weight_arrays, axis=0)

    # 使用array_to_weights函数将聚合后的权重数组转换回权重字典。
    aggregated_weights = array_to_weights(aggregated_weight_array, weight_dicts[0])

    # 返回聚合后的权重字典和最好的客户端索引列表。
    return aggregated_weights, best_clients_indices

## 1.3定义服务端

In [8]:
def train_server(fx_client, y, l_epoch_count, l_epoch, idx, len_batch):
    """

    Args:
        fx_client: 一个函数，用于在客户端更新模型参数，它接受以下参数：net_model_client（客户端模型），optimizer_client（客户端优化器），train_loader（客户端训练数据），l_epoch（客户端训练轮数）。
        y:目标变量的标签值。
        l_epoch_count:训练的总轮数
        l_epoch:当前训练的轮数
        idx:用于选择在全局模型中使用哪些本地模型进行更新的客户端的索引。
        len_batch:训练数据的批次大小。

    Returns:

    """
    # 这些是全局变量，因为它们在函数内被更新，并且在函数之外被调用。
    """
    net_model_server: 全局模型。
    criterion: 损失函数，用于计算模型的误差。
    optimizer_server: 优化器，用于更新全局模型的参数。
    device: 设备（CPU或GPU）用于计算。
    batch_acc_train: 当前批次的准确度。
    batch_loss_train: 当前批次的损失。
    l_epoch_check: 在训练期间用于检查损失和准确度的训练周期数。
    fed_check: 用于检查训练周期是否已完成的标志。
    loss_train_collect: 用于收集所有客户端训练损失的列表。
    acc_train_collect: 用于收集所有客户端训练准确度的列表。
    count1: 计数器，用于跟踪当前已经训练的客户端数量。
    acc_avg_all_user_train: 所有客户端训练准确度的平均值。
    loss_avg_all_user_train: 所有客户端训练损失的平均值。
    idx_collect: 用于跟踪已经训练的客户端的索引列表。
    w_locals_server: 所有客户端本地模型参数的列表。
    w_glob_server: 全局模型参数的列表。
    net_server: 全局模型。
    """
    global net_model_server, criterion, optimizer_server, device, batch_acc_train, batch_loss_train, l_epoch_check, fed_check
    global loss_train_collect, acc_train_collect, count1, acc_avg_all_user_train, loss_avg_all_user_train, idx_collect, w_locals_server, w_glob_server, net_server
    global loss_train_collect_user, acc_train_collect_user, lr

    # net_server是全局模型，返回制定索引的本地模型
    net_server = copy.deepcopy(net_model_server[idx]).to(
        device)  # copy.deepcopy() 函数用于创建一个当前本地模型的副本，以便我们可以在全局模型的更新过程中使用它，而不会对原始本地模型进行更改。
    # 方法将模型设置为训练模式，这意味着在计算时会使用训练期间的正则化技术，如dropout或batch normalization。
    net_server.train()
    # 是一个PyTorch中的Adam优化器的实现，它接受模型参数和学习率作为参数，用于更新模型参数以最小化损失函数。在这里，我们使用全局模型的参数和一个预定义的学习率 lr 创建了一个Adam优化器对象
    optimizer_server = torch.optim.Adam(net_server.parameters(), lr=lr)

    # 1.train and update
    # 用于清空之前的梯度信息，这样我们可以在每个训练迭代中计算新的梯度并更新模型参数。
    optimizer_server.zero_grad()

    fx_client = fx_client.to(device)
    y = y.to(device)

    # ---------forward prop-------------
    fx_server = net_server(fx_client)  # 作为输入传递到全局模型 net_server 中，然后返回模型的预测输出 fx_server

    # calculate loss
    loss = criterion(fx_server, y)
    # calculate accuracy
    acc = calculate_accuracy(fx_server, y)

    # --------backward prop--------------
    loss.backward()
    # 由于我们需要在全局模型更新之前将 fx_client 更新到最新的版本，因此我们使用 clone().detach() 函数来创建一个新的 dfx_client 张量，它具有相同的值但不会被计算图所记录。
    dfx_client = fx_client.grad.clone().detach()
    optimizer_server.step()

    batch_loss_train.append(loss.item())
    batch_acc_train.append(acc.item())

    # Update the server-side model for the current batch
    net_model_server[idx] = copy.deepcopy(net_server)

    # count1: to track the completion of the local batch associated with one client
    count1 += 1
    if count1 == len_batch:
        acc_avg_train = sum(batch_acc_train) / len(batch_acc_train)  # 计算当前batch的准确率
        loss_avg_train = sum(batch_loss_train) / len(batch_loss_train)  # 计算当前batch的损失

        batch_acc_train = []  # 将当前batch准确率清零
        batch_loss_train = []
        count1 = 0

        prRed('Client{} Train => Local Epoch: {} \tAcc: {:.3f} \tLoss: {:.4f}'.format(idx, l_epoch_count, acc_avg_train,
                                                                                      loss_avg_train))

        # copy the last trained model in the batch
        # 的状态字典复制到一个新的字典中，以便我们可以将其发送到参与者，从而启动下一轮的联邦学习。注意，w_server 中包含的参数是最新一轮训练的参数，因此每个参与者将从这些参数开始训练它们的本地模型。
        w_server = net_server.state_dict()

        # If one local epoch is completed, after this a new client will come
        if l_epoch_count == l_epoch - 1:
            # l_epoch_count 是本地epoch的计数器，l_epoch 是本地epoch的总数。当计数器 l_epoch_count 等于总数 l_epoch 减 1 时，说明本地epoch已经完成。
            # # 标记已经完成本地epoch
            l_epoch_check = True  # to evaluate_server function - to check local epoch has completed or not
            # We store the state of the net_glob_server()
            # w_server 是全局模型中最新的训练参数，w_locals_server 是用于存储每个参与者的最后一轮训练参数的列表。因此，当本地epoch完成时，将 w_server 添加到 w_locals_server 中，以便之后将其发送到联邦平均服务器。
            w_locals_server.append(copy.deepcopy(w_server))

            # we store the last accuracy in the last batch of the epoch and it is not the average of all local epochs
            # this is because we work on the last trained model and its accuracy (not earlier cases)

            # print("accuracy = ", acc_avg_train)
            acc_avg_train_all = acc_avg_train  # 记录最后一个batch的准确率和损失，作为本地epoch的结果
            loss_avg_train_all = loss_avg_train  #

            # accumulate accuracy and loss for each new user
            loss_train_collect_user.append(loss_avg_train_all)  # 将本地epoch的损失添加到损失列表中
            acc_train_collect_user.append(acc_avg_train_all)  # # 将本地epoch的准确率添加到准确率列表中

            # collect the id of each new user
            if idx not in idx_collect:
                idx_collect.append(idx)
                # print(idx_collect)
#                 print("已经训练的客户端:" + str(idx_collect))

        # This is for federation process--------------------
        if len(idx_collect) == num_users * frac:
            # 如果客户端编号列表的长度等于客户端总数，说明所有客户端的训练结果都已经到达服务器了。
            # 这里不对，是选择的客户端总数
            fed_check = True  # to evaluate_server function  - to check fed check has hitted

            # 异常检测
            # 使用异常检测移除异常客户端
            # anomaly_threshold = 2  # 自定义阈值
            # w_locals_server = remove_anomalies(w_locals_server, w_glob_server.state_dict(), anomaly_threshold)

            # ================== 使用不同的聚合算法 =================
            w_locals_server,smallest_cluster_indices = remove_anomalies_kmeans(w_locals_server, w_glob_server, n_clusters=4
            , remove_ratio=poisoned_frac)

            # ================== 使用不同的聚合算法 =================
            w_glob_server = FedAvg(w_locals_server)  # 使用联邦平均算法更新全局模型，将所有客户端的本地模型参数传入该函数中。
            # num_to_select = int(num_users * (1 - poisoned_frac - 0.1))  # 选择的客户端数量
            # w_glob_server, selected_clients_indices = krum_aggregation(w_locals_server, num_to_select)

            # server-side global model update and distribute that model to all clients ------------------------------
            net_glob_server.load_state_dict(w_glob_server)  # 将更新后的全局模型参数加载到服务器端的模型中。
            net_model_server = [net_glob_server for i in
                                range(num_users)]  # 创建一个长度为客户端数量的列表，每个元素都是更新后的全局模型。这个列表用于向每个客户端分发全局模型参数。



            acc_avg_all_user_train = sum(acc_train_collect_user) / len(acc_train_collect_user)  # 计算所有客户端训练结果的平均准确率和损失
            loss_avg_all_user_train = sum(loss_train_collect_user) / len(loss_train_collect_user)

            # 更新性能指标列表
            loss_train_collect.append(loss_avg_all_user_train)
            acc_train_collect.append(acc_avg_all_user_train)

            acc_train_collect_user = []
            loss_train_collect_user = []

            w_locals_server = []  # # 清空本地模型参数列表
            idx_collect = []  # 清空客户端编号列表

    # send gradients to the client
    return dfx_client


def evaluate_server(fx_client, y, idx, len_batch, ell, selected_clients):
    global net_model_server, criterion, batch_acc_test, batch_loss_test, check_fed, net_server, net_glob_server
    global loss_test_collect, acc_test_collect, count2, num_users, acc_avg_train_all, loss_avg_train_all, w_glob_server, l_epoch_check, fed_check
    global loss_test_collect_user, acc_test_collect_user, acc_avg_all_user_train, loss_avg_all_user_train

    net = copy.deepcopy(net_model_server[idx]).to(device)
    net.eval()
    return_local_results = False

    with torch.no_grad():
        # with torch.no_grad()是一个上下文管理器，它可以暂时关闭所有的requires_grad标志，从而不计算梯度1。这样可以节省内存，提高推理速度，也可以避免不必要的梯度累积2。通常在验证或部署模型时使用这个方法3。
        fx_client = fx_client.to(device)
        y = y.to(device)
        # ---------forward prop-------------
        fx_server = net(fx_client)

        # calculate loss
        loss = criterion(fx_server, y)
        # calculate accuracy
        acc = calculate_accuracy(fx_server, y)

        batch_loss_test.append(loss.item())
        batch_acc_test.append(acc.item())

        count2 += 1
        if count2 == len_batch:
            acc_avg_test = sum(batch_acc_test) / len(batch_acc_test)
            loss_avg_test = sum(batch_loss_test) / len(batch_loss_test)

            batch_acc_test = []
            batch_loss_test = []
            count2 = 0

            prGreen('Client{} Test =>                   \tAcc: {:.3f} \tLoss: {:.4f}'.format(idx, acc_avg_test,
                                                                                             loss_avg_test))

            # if a local epoch is completed
            if l_epoch_check:
                l_epoch_check = False
                return_local_results = True

                # Store the last accuracy and loss
                acc_avg_test_all = acc_avg_test
                loss_avg_test_all = loss_avg_test

                loss_test_collect_user.append(loss_avg_test_all)
                acc_test_collect_user.append(acc_avg_test_all)

            # if federation is happened----------
            if fed_check:
                fed_check = False
                print("------------------------------------------------")
                print("------ Federation process at Server-Side ------- ")
                print("------------------------------------------------")

                # 计算Krum选定客户端的平均准确率和损失
                if selected_clients is None or len(selected_clients) == 0:
                    acc_avg_all_user = sum(acc_test_collect_user) / len(acc_test_collect_user)
                    loss_avg_all_user = sum(loss_test_collect_user) / len(loss_test_collect_user)
                else:
                    print("选择的客户端index:", selected_clients)
                    acc_test_collect_user = [acc_test_collect_user[i] for i in selected_clients]
                    loss_test_collect_user = [loss_test_collect_user[i] for i in selected_clients]

                    acc_avg_all_user = sum(acc_test_collect_user) / len(acc_test_collect_user)
                    loss_avg_all_user = sum(loss_test_collect_user) / len(loss_test_collect_user)

                loss_test_collect.append(loss_avg_all_user)
                acc_test_collect.append(acc_avg_all_user)
                acc_test_collect_user = []
                loss_test_collect_user = []

                print("====================== SERVER V1==========================")
                print(' Train: Round {:3d}, Avg Accuracy {:.3f} | Avg Loss {:.3f}'.format(ell, acc_avg_all_user_train,
                                                                                          loss_avg_all_user_train))
                print(' Test: Round {:3d}, Avg Accuracy {:.3f} | Avg Loss {:.3f}'.format(ell, acc_avg_all_user,
                                                                                         loss_avg_all_user))
                print("==========================================================")

    if return_local_results:
        return acc_avg_test_all, loss_avg_test_all
    else:
        return None, None


class Client(object):
    def __init__(self, net_client_model, idx, lr, device, dataset_train=None, dataset_test=None, idxs=None,
                 idxs_test=None, is_attacker=None, batch_size=128):
        """
        :param idxs: idxs是一个表示该客户端用于训练的数据集的索引列表。在联邦学习中，原始数据集通常由多个客户端持有，每个客户端只能访问自己所持有的部分数据集。因此，为了让每个客户端只使用自己所持有的数据进行训练，需要将原始数据集划分成多个部分，每个部分由一个客户端持有，并通过idxs将该客户端用于训练的数据集的索引列表传递给Client类的构造函数。
        :param idxs_test:
        """
        # net_client_model:一个与客户端实例相关的神经网络模型。
        self.batch_size = batch_size
        self.is_attacker = is_attacker
        self.idx = idx  # 一个整数，表示客户端的索引
        self.device = device  # 一个字符串，表示执行客户端计算的设备。
        self.lr = lr
        self.local_ep = 1
        # self.selected_clients = []
        # DatasetSplit(dataset_train, idxs)表示使用DatasetSplit类将原始的数据集dataset_train按照索引idxs进行划分，以获得当前客户端可用于训练的数据集。
        self.ldr_train = DataLoader(DatasetSplit(dataset_train, idxs), batch_size=self.batch_size,
                                    shuffle=True)
        self.ldr_test = DataLoader(DatasetSplit(dataset_test, idxs_test), batch_size=self.batch_size, shuffle=True)

    def train(self, net):
        net.train()
        optimizer_client = torch.optim.Adam(net.parameters(), lr=self.lr)

        for iter in range(self.local_ep):
            if self.is_attacker:
                dataset_split = self.ldr_train.dataset
                num_malicious_samples = len(dataset_split.idxs)
                dataset_split.add_malicious_samples(num_malicious_samples,dataset_type=dataset_choice)

                # Refresh the DataLoader after adding malicious samples
                self.ldr_train = DataLoader(dataset_split, batch_size=self.batch_size, shuffle=True)

            # 外层循环是客户端的本地训练轮数self.local_ep
            len_batch = len(self.ldr_train)  # 计算该客户端的训练集数据分成的批次数。
            for batch_idx, (images, labels) in enumerate(self.ldr_train):
                # 内层循环是数据加载器self.ldr_train中每个批次的训练。在每个批次中，将图像和标签加载到设备上，然后将优化器的梯度清零。
                images, labels = images.to(self.device), labels.to(self.device)
                optimizer_client.zero_grad()
                # ---------forward prop-------------
                fx = net(images)
                # 生成一个可求导的副本client_fx
                client_fx = fx.clone().detach().requires_grad_(True)

                # Sending activations to server and receiving gradients from server
                dfx = train_server(client_fx, labels, iter, self.local_ep, self.idx, len_batch)

                # --------backward prop -------------
                fx.backward(dfx)
                optimizer_client.step()

            # prRed('Client{} Train => Epoch: {}'.format(self.idx, ell))

        return net.state_dict()

    def evaluate(self, net=None, ell=None, selected_clients=None):
        net.eval()

        with torch.no_grad():
            len_batch = len(self.ldr_test)
            for batch_idx, (images, labels) in enumerate(self.ldr_test):
                images, labels = images.to(self.device), labels.to(self.device)
                # ---------forward prop-------------
                fx = net(images)

                # Sending activations to server
                acc_avg_test_all, loss_avg_test_all = evaluate_server(fx, labels, self.idx, len_batch, ell,
                                                                      selected_clients)

            # prRed('Client{} Test => Epoch: {}'.format(self.idx, ell))
            if loss_avg_test_all is not None and acc_avg_test_all is not None:
                self.loss_avg_test_all = loss_avg_test_all
                self.acc_avg_test_all = acc_avg_test_all

        return

# 2.Data Loading

In [9]:
from glob import glob
from sklearn.model_selection import train_test_split
from torchvision import transforms
from torch.utils.data import Dataset
from PIL import Image, ImageFilter
from PIL import ImageEnhance

class DatasetSplit(Dataset):
    def __init__(self, dataset, idxs):
        self.dataset = dataset
        self.idxs = list(idxs)

    def __len__(self):
        return len(self.idxs)

    def __getitem__(self, item):
        image, label = self.dataset[self.idxs[item]]
        return image, label

    def add_malicious_samples(self, num_malicious_samples, dataset_type='CIFAR10'):
        for i in range(num_malicious_samples):
            # Randomly select an image to modify
            dataset_idx = random.choice(self.idxs)

            # Access the image data depending on the dataset structure
            img = self.dataset.data[dataset_idx]
            img = Image.fromarray(img)

            # Apply a random modification to the image (e.g., flip, rotate, or change color)
            modified_img = img.transpose(Image.FLIP_LEFT_RIGHT)  # Flip the image horizontally
            modified_img = modified_img.rotate(random.randint(0, 360))  # Rotate the image randomly

            brightness = ImageEnhance.Brightness(modified_img)
            modified_img = brightness.enhance(random.uniform(0.5, 1.5))

            contrast = ImageEnhance.Contrast(modified_img)
            modified_img = contrast.enhance(random.uniform(0.5, 1.5))

            saturation = ImageEnhance.Color(modified_img)
            modified_img = saturation.enhance(random.uniform(0.5, 1.5))

            # Add additional distortions
            modified_img = modified_img.filter(
                ImageFilter.GaussianBlur(radius=random.uniform(0, 2)))  # Apply Gaussian blur

            # Add noise to the image
            noise = np.random.normal(0, 25, modified_img.size)
            noise = noise.reshape(modified_img.size[::-1]).T.astype(np.uint8)
            modified_img = Image.fromarray(np.array(modified_img) + noise)

            # Clip the pixel values to be within the valid range (0-255)
            modified_img = np.clip(modified_img, 0, 255)

            # Convert the modified PIL Image back to a numpy array
            modified_img = np.array(modified_img)

            self.dataset.data[dataset_idx] = modified_img




# ==============================================================
# Custom dataset prepration in Pytorch format
class SkinData(Dataset):
    def __init__(self, df, data, targets, transform=None):
        self.df = df
        self.data = data
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        img, target = self.data[index], self.targets[index]
        img = Image.fromarray(img)

        if self.transform is not None:
            img = self.transform(img)


        return img, target


def dataset_iid(dataset, num_users):
    """
    该函数接受一个数据集dataset和一个整数num_users作为输入。
    它的作用是将数据集分割成num_users份，以便每个客户端都有一份相同分布的数据集。
    :param dataset:
    :param num_users:
    :return:函数返回一个字典dict_users，其中包含num_users个键，每个键对应一个客户端，值为该客户端所分配的数据集索引的集合。
    dict_users:{idx:int : []:list}
    """
    # 该函数首先计算每个客户端应该拥有的数据量num_items
    num_items = int(len(dataset)/num_users)
    dict_users, all_idxs = {}, [i for i in range(len(dataset))]
    for i in range(num_users):
        # 接着，函数使用np.random.choice函数从all_idxs中选择num_items个索引，将这些索引添加到字典dict_users的第i个键中，表示第i个客户端的数据集。在选择后，从all_idxs中移除已经分配给第i个客户端的索引。
        dict_users[i] = set(np.random.choice(all_idxs, num_items, replace = False))
        all_idxs = list(set(all_idxs) - dict_users[i])
    return dict_users

def load_data(dataset_choice, num_users):
    """

    :param dataset_choice: 选择的数据集
    :param num_users: 用户的数量
    :return:
    """
    if dataset_choice == 'HAM10000':
        df = pd.read_csv('data/HAM10000_metadata.csv')
        print(df.head())

        lesion_type = {
            'nv': 'Melanocytic nevi',
            'mel': 'Melanoma',
            'bkl': 'Benign keratosis-like lesions ',
            'bcc': 'Basal cell carcinoma',
            'akiec': 'Actinic keratoses',
            'vasc': 'Vascular lesions',
            'df': 'Dermatofibroma'
        }

        # merging both folders of HAM1000 dataset -- part1 and part2 -- into a single directory
        imageid_path = {os.path.splitext(os.path.basename(x))[0]: x
                        for x in glob(os.path.join("data", '*', '*.jpg'))}

        # print("path---------------------------------------", imageid_path.get)
        # 将图像id映射为图像文件的路径，并将其存储在数据集中的path列中。
        df['path'] = df['image_id'].map(imageid_path.get)
        # 将诊断编码映射为对应的分类名称，并将其存储在数据集中的cell_type列中。
        df['cell_type'] = df['dx'].map(lesion_type.get)
        # 将分类名称转换为数字编码，并将其存储在数据集中的target列中。这里使用了.
        # 可以将字符串类型的分类变量转换为数字编码，其中不同的分类名称对应不同的数字编码。
        df['target'] = pd.Categorical(df['cell_type']).codes
        print(df['cell_type'].value_counts())
        print(df['target'].value_counts())

        # =============================================================================
        # Train-test split
        train, test = train_test_split(df, test_size=0.2)

        train = train.reset_index()

        test = test.reset_index()

        # Load image data and targets
        image_data = []
        for path in train['path']:
            image = Image.open(path).resize((64, 64))
            image_data.append(np.array(image))
        train_data = np.array(image_data)
        train_targets = train['target'].astype(np.int64).to_numpy()  # Change dtype to int64

        image_data = []
        for path in test['path']:
            image = Image.open(path).resize((64, 64))
            image_data.append(np.array(image))
        test_data = np.array(image_data)
        test_targets = test['target'].astype(np.int64).to_numpy()  # Change dtype to int64



        # =============================================================================
        #                         Data preprocessing
        # =============================================================================
        # Data preprocessing: Transformation
        mean = [0.485, 0.456, 0.406]
        std = [0.229, 0.224, 0.225]

        train_transforms = transforms.Compose([transforms.RandomHorizontalFlip(),
                                               transforms.RandomVerticalFlip(),
                                               transforms.Pad(3),
                                               transforms.RandomRotation(10),
                                               transforms.CenterCrop(64),
                                               transforms.ToTensor(),
                                               transforms.Normalize(mean=mean, std=std)
                                               ])

        test_transforms = transforms.Compose([
            transforms.Pad(3),
            transforms.CenterCrop(64),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std)
        ])

        # With augmentation
        dataset_train = SkinData(train, train_data, train_targets, transform=train_transforms)
        dataset_test = SkinData(train, test_data, test_targets, transform=test_transforms)

        # ----------------------------------------------------------------
        dict_users = dataset_iid(dataset_train, num_users)
        dict_users_test = dataset_iid(dataset_test, num_users)
    elif dataset_choice == 'CIFAR10':
        # =============================================================================
        #                         Data loading
        # =============================================================================
        # Load CIFAR-10 dataset
        trainset = datasets.CIFAR10(root='./data', train=True, download=True)
        testset = datasets.CIFAR10(root='./data', train=False, download=True)

        train_df = pd.DataFrame(trainset.targets, columns=['target'])
        test_df = pd.DataFrame(testset.targets, columns=['target'])

        # Set the class names
        class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

        train_df['cell_type'] = train_df['target'].apply(lambda x: class_names[x])
        test_df['cell_type'] = test_df['target'].apply(lambda x: class_names[x])

        print(train_df['cell_type'].value_counts())
        print(train_df['target'].value_counts())

        # =============================================================================
        # Train-test split
        train = train_df.reset_index()
        test = test_df.reset_index()

        # =============================================================================
        #                         Data preprocessing
        # =============================================================================
        # Data preprocessing: Transformation
        mean = [0.485, 0.456, 0.406]
        std = [0.229, 0.224, 0.225]

        train_transforms = transforms.Compose([transforms.RandomHorizontalFlip(),
                                               transforms.RandomVerticalFlip(),
                                               transforms.Pad(3),
                                               transforms.RandomRotation(10),
                                               transforms.CenterCrop(32),
                                               transforms.ToTensor(),
                                               transforms.Normalize(mean=mean, std=std)
                                               ])

        test_transforms = transforms.Compose([
            transforms.Pad(3),
            transforms.CenterCrop(32),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std)
        ])

        # With augmentation
        dataset_train = datasets.CIFAR10(root='./data', train=True, transform=train_transforms, download=True)
        dataset_test = datasets.CIFAR10(root='./data', train=False, transform=test_transforms, download=True)

        # ----------------------------------------------------------------
        dict_users = dataset_iid(dataset_train, num_users)
        dict_users_test = dataset_iid(dataset_test, num_users)
    else:
        raise ValueError("Invalid dataset_choice: Choose either 'HAM10000' or 'CIFAR10'")

    return dataset_train,dataset_test,dict_users,dict_users_test

In [10]:
dataset_train, dataset_test, dict_users, dict_users_test = load_data(dataset_choice, num_users)


Files already downloaded and verified
Files already downloaded and verified
frog          5000
truck         5000
deer          5000
automobile    5000
bird          5000
horse         5000
ship          5000
cat           5000
dog           5000
airplane      5000
Name: cell_type, dtype: int64
6    5000
9    5000
4    5000
1    5000
2    5000
7    5000
8    5000
3    5000
5    5000
0    5000
Name: target, dtype: int64
Files already downloaded and verified
Files already downloaded and verified


## 2.3数据投毒——标签翻转

In [11]:
def replace_label1_with_label2_on_df(df,label1,label2,poisoned_dict_users):
    """
    标签反转
    :param df:dataframe
    :param label1: 等待翻转的标签
    :param label2: 需要翻转的标签
    :param poisoned_dict_users:包含索引列表的字典,中毒的用户
    :return:
    """
    for idx_list in poisoned_dict_users.values():
        for idx in idx_list:
            if df.loc[idx,'target'] == label1:
                df.loc[idx,'target'] = label2
    # df.loc[df['target'] == label1, 'target'] = label2
    return df

def random_select_poisoning_users(dict_users, n):
    """
    随机选择n个key-value对
    :param dict_users: 字典，key为索引，value为包含索引值的列表
    :param n: 选择的key-value对的数量
    :return: 随机选择的key-value对组成的字典
    """
    selected = {}
    keys = random.sample(list(dict_users.keys()), n)
    for k in keys:
        selected[k] = dict_users[k]
    return selected

# def poison_data(dataset_train, dict_users, poisoned_users_num, original_label, target_label):
#     poisoned_dict_users = random_select_poisoning_users(dict_users, poisoned_users_num)
#     replace_label1_with_label2_on_df(dataset_train.df, original_label, target_label, poisoned_dict_users)
#     return poisoned_dict_users

def print_poisoning_results(poisoned_dict_users, dataset_train):
    for poisoned_user_key in poisoned_dict_users:
        print("被投毒的用户:", poisoned_user_key)

    print("标签反转后的target统计:")
    print(dataset_train.df['target'].value_counts())

def cifar10_to_dataframe(dataset):
    data = [dataset[i] for i in range(len(dataset))]
    images, labels = zip(*data)
    df = pd.DataFrame({"image": images, "target": labels})
    return df



def poison_data(dataset_train, dataset_choice, dict_users, poisoned_users_num, label_mappings):
    if dataset_choice == 'CIFAR10':
        df_train = cifar10_to_dataframe(dataset_train)
    elif dataset_choice == 'HAM10000':
        df_train = dataset_train.df
    else:
        raise ValueError("Invalid dataset choice.")

    poisoned_dict_users = random_select_poisoning_users(dict_users, poisoned_users_num)

    for original_label, target_label in label_mappings:
        replace_label1_with_label2_on_df(df_train, original_label, target_label, poisoned_dict_users)

    # 修改 dataset_train 的标签
    if dataset_choice == 'CIFAR10':
        for idx, row in df_train.iterrows():
            dataset_train.targets[idx] = row['target']

    return poisoned_dict_users

def poison_data_random(dataset_train, dataset_choice, dict_users, poisoned_users_num):
    if dataset_choice == 'CIFAR10':
        df_train = cifar10_to_dataframe(dataset_train)
    elif dataset_choice == 'HAM10000':
        df_train = dataset_train.df
    else:
        raise ValueError("Invalid dataset choice.")

    poisoned_dict_users = random_select_poisoning_users(dict_users, poisoned_users_num)

    for idx_list in poisoned_dict_users.values():
        for idx in idx_list:
            labels = list(set(df_train['target'].unique()) - {df_train.loc[idx, 'target']})
            df_train.loc[idx, 'target'] = random.choice(labels)

    # 修改 dataset_train 的标签
    if dataset_choice == 'CIFAR10':
        for idx, row in df_train.iterrows():
            dataset_train.targets[idx] = row['target']

    return poisoned_dict_users

In [12]:
# =============================================================================
#                         Poisoning
# =============================================================================

def generate_label_mappings(dataset_choice):
    if dataset_choice == 'CIFAR10':
        num_classes = 10
    elif dataset_choice == 'HAM10000':
        num_classes = 7
    else:
        raise ValueError("Invalid dataset choice.")

    label_mappings = []
    for i in range(num_classes):
        target_label = random.choice([j for j in range(num_classes) if j != i])
        label_mappings.append((i, target_label))

    return label_mappings

# 定义攻击模式和目标标签
# attack_pattern = np.full((32, 32, 3), 255, dtype=np.uint8)
# target_label = 5

poisoned_users_num = int(poisoned_frac * num_users)
# label_mappings = [(4, 2), (1, 7), (3, 1), (2, 6), (6, 5)]
label_mappings = generate_label_mappings(dataset_choice)
poisoned_dict_users = poison_data(dataset_train, dataset_choice, dict_users, poisoned_users_num, label_mappings)
# # poisoned_dict_users = poison_data_model(dataset_train, dataset_choice, dict_users, poisoned_users_num, attack_pattern, target_label)
for poisoned_user_key in poisoned_dict_users:
    print("被投毒的用户:", poisoned_user_key)

# 如果使用的是 HAM10000 数据集，您可以直接使用 dataset_train.df 查看标签分布
if dataset_choice == 'HAM10000':
    print("标签反转后的target统计:")
    print(dataset_train.df['target'].value_counts())
elif dataset_choice == 'CIFAR10':
    cifar10_df = cifar10_to_dataframe(dataset_train)
    print("标签反转后的target统计:")
    print(cifar10_df['target'].value_counts())

被投毒的用户: 0
被投毒的用户: 19
标签反转后的target统计:
0    6550
1    6001
3    5487
5    4979
4    4522
6    4509
7    4505
2    4489
8    4486
9    4472
Name: target, dtype: int64


# 3.模型train和test
## 3.1 初始化

In [13]:
# ------------ Training And Testing  -----------------
net_glob_client.train()
# copy weights
w_glob_client = net_glob_client.state_dict()
# Federation takes place after certain local epochs in train() client-side
# this epoch is global epoch, also known as rounds

total_time = 0.0  # 初始化总时间为0
best_clients_indices = None
krum_acc_test_collect = []
krum_loss_test_collect = []

In [14]:
for iter in range(epochs):
    start_time = time.time()  # 记录开始时间
    m = max(int(frac * num_users), 1)
    idxs_users = np.random.choice(range(num_users), m, replace=False)  # ，replace=False表示不允许重复选择。
    w_locals_client = []  # 用于存储每个客户端训练后的本地模型参数。
    loss_avg_test_all_dict = {}
    acc_avg_test_all_dict = {}
    idx_local_client_dict = {}

    for idx in idxs_users:
        local = Client(net_glob_client, idx, lr, device, dataset_train=dataset_train, dataset_test=dataset_test,
                       idxs=dict_users[idx], idxs_test=dict_users_test[idx], is_attacker=None)
        # Training ------------------
        w_client = local.train(net=copy.deepcopy(net_glob_client).to(device))
        idx_local_client_dict[len(w_locals_client)] = idx
        w_locals_client.append(copy.deepcopy(w_client))

        # Testing -------------------
        local.evaluate(net=copy.deepcopy(net_glob_client).to(device), ell=iter, selected_clients=best_clients_indices)

        # Update the dictionaries with the client's self.loss_avg_test_all and self.acc_avg_test_all
        loss_avg_test_all_dict[idx] = local.loss_avg_test_all
        acc_avg_test_all_dict[idx] = local.acc_avg_test_all

    print("idxs_users",idxs_users)
    # Ater serving all clients for its local epochs------------
    # Federation process at Client-Side------------------------
    print("------------------------------------------------------------")
    print("------ Fed Server: Federation process at Client-Side -------")
    print("------------------------------------------------------------")
    # w_locals_client是所有客户端训练后的本地模型参数列表，FedAvg函数是加权平均函数，返回全局模型参数w_glob_client。
    # w_glob_client = FedAvg(w_locals_client)
    # # 或使用基于聚类的方法
    w_locals_client, smallest_cluster_indices = remove_anomalies_kmeans(w_locals_client, w_glob_client, n_clusters=4,
                                                                        remove_ratio=poisoned_frac)
    w_glob_client = FedAvg(w_locals_client)
    best_clients_indices = [client for client in list(range(len(idxs_users))) if client not in smallest_cluster_indices]
    # 调用 Krum 算法
    # num_to_select = int(num_users * (1 - poisoned_frac - 0.1))  # 选择的客户端数量
    # w_glob_client, best_clients_indices = krum_aggregation(w_locals_client, num_to_select)
    print([idxs_users[i] for i in best_clients_indices])
    # Update client-side global model
    net_glob_client.load_state_dict(w_glob_client)

    print("fedserver选择的客户端index:", best_clients_indices)
    best_clients_idxs = [idx_local_client_dict[i] for i in best_clients_indices]
    krum_acc_test_collect_user = [acc_avg_test_all_dict[i] for i in best_clients_idxs]
    for acc in krum_acc_test_collect_user:
        print("acc:",acc)
    krum_loss_test_collect_user = [loss_avg_test_all_dict[i] for i in best_clients_idxs]
    
    krum_acc_avg_all_user = sum(krum_acc_test_collect_user) / len(krum_acc_test_collect_user)
    krum_loss_avg_all_user = sum(krum_loss_test_collect_user) / len(krum_loss_test_collect_user)
    krum_acc_test_collect.append(krum_acc_avg_all_user)
    krum_loss_test_collect.append(krum_loss_avg_all_user)

    print("====================== Fed Server==========================")
    print(' Train: Round {:3d}, Avg Accuracy {:.3f} | Avg Loss {:.3f}'.format(iter, acc_avg_all_user_train,
                                                                              loss_avg_all_user_train))
    print(' Test: Round {:3d}, Avg Accuracy {:.3f} | Avg Loss {:.3f}'.format(iter, krum_acc_avg_all_user,
                                                                             krum_loss_avg_all_user))
    print("==========================================================")

    end_time = time.time()  # 记录结束时间
    epoch_time = end_time - start_time  # 计算epoch所耗费的时间
    total_time += epoch_time  # 将时间差加到总时间中
    # 将时间差值转换为小时、分钟和秒数
    hours, rem = divmod(epoch_time, 3600)
    minutes, seconds = divmod(rem, 60)
    print(f"Epoch {iter} finished in {int(hours):02d}:{int(minutes):02d}:{int(seconds):02d}")
    print(f"Epoch {iter} finished. Total time: {total_time:.2f} seconds")

 Client11 Train => Local Epoch: 0 	Acc: 24.616 	Loss: 2.0384
 Client11 Test =>                   	Acc: 7.779 	Loss: 2.9855
 Client4 Train => Local Epoch: 0 	Acc: 24.079 	Loss: 2.0534
 Client4 Test =>                   	Acc: 7.738 	Loss: 3.0779
 Client14 Train => Local Epoch: 0 	Acc: 23.081 	Loss: 2.0679
 Client14 Test =>                   	Acc: 11.335 	Loss: 2.9693
 Client18 Train => Local Epoch: 0 	Acc: 22.833 	Loss: 2.0726
 Client18 Test =>                   	Acc: 10.419 	Loss: 2.8585
 Client8 Train => Local Epoch: 0 	Acc: 22.987 	Loss: 2.0482
 Client8 Test =>                   	Acc: 8.897 	Loss: 2.9100
 Client1 Train => Local Epoch: 0 	Acc: 22.119 	Loss: 2.0718
 Client1 Test =>                   	Acc: 8.425 	Loss: 3.3848
 Client12 Train => Local Epoch: 0 	Acc: 22.403 	Loss: 2.0721
 Client12 Test =>                   	Acc: 7.334 	Loss: 2.8268
 Client6 Train => Local Epoch: 0 	Acc: 24.350 	Loss: 2.0530
 Client6 Test =>                   	Acc: 11.887 	Loss: 2.9331
 Client16 Train => Lo

c:\Users\Arthur\anaconda3\envs\torch\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


 Client9 Test =>                   	Acc: 10.695 	Loss: 2.8730
------------------------------------------------
------ Federation process at Server-Side ------- 
------------------------------------------------
====================== SERVER V1==========================
 Train: Round   0, Avg Accuracy 25.383 | Avg Loss 2.004
 Test: Round   0, Avg Accuracy 9.702 | Avg Loss 2.974
idxs_users [11  4 14 18  8  1 12  6 16 17  0  7 10  5  3 13 19  2 15  9]
------------------------------------------------------------
------ Fed Server: Federation process at Client-Side -------
------------------------------------------------------------


c:\Users\Arthur\anaconda3\envs\torch\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


[4, 14, 18, 8, 1, 12, 6, 16, 17, 0, 7, 10, 5, 3, 13, 19, 2, 15]
fedserver选择的客户端index: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
acc: 7.738415956497192
acc: 11.334859848022461
acc: 10.418911695480347
acc: 8.896821022033691
acc: 8.425377130508423
acc: 7.334321022033691
acc: 11.887122869491577
acc: 9.617456912994385
acc: 10.203394412994385
acc: 9.287446022033691
acc: 8.661099195480347
acc: 11.025053977966309
acc: 10.048491477966309
acc: 9.577047348022461
acc: 9.341325402259827
acc: 10.594019412994385
acc: 10.479525804519653
acc: 10.695043087005615
====================== Fed Server==========================
 Train: Round   0, Avg Accuracy 25.383 | Avg Loss 2.004
 Test: Round   0, Avg Accuracy 9.754 | Avg Loss 2.979
Epoch 0 finished in 00:02:30
Epoch 0 finished. Total time: 150.37 seconds
 Client10 Train => Local Epoch: 0 	Acc: 31.758 	Loss: 1.8261
 Client10 Test =>                   	Acc: 15.073 	Loss: 2.7023
 Client5 Train => Local Epoch: 0 	Acc: 30.662 	Loss: 1.8557

...

[3, 15, 11, 12, 17, 10, 14, 16, 1, 13, 7, 5, 4, 8, 9, 2, 18, 6]
fedserver选择的客户端index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 11, 12, 13, 14, 15, 16, 17, 18, 19]
acc: 71.42376136779785
acc: 66.37931060791016
acc: 72.2992992401123
acc: 59.011314392089844
acc: 71.1274242401123
acc: 66.37931060791016
acc: 64.91783332824707
acc: 71.3227367401123
acc: 68.74326515197754
acc: 69.32920265197754
acc: 65.71928787231445
acc: 69.21470832824707
acc: 64.32516193389893
acc: 69.10021591186523
acc: 66.28502082824707
acc: 64.26454734802246
acc: 65.28825378417969
acc: 76.83863067626953
====================== Fed Server==========================
 Train: Round  98, Avg Accuracy 77.465 | Avg Loss 0.706
 Test: Round  98, Avg Accuracy 67.887 | Avg Loss 1.019
Epoch 98 finished in 00:02:00
Epoch 98 finished. Total time: 11970.96 seconds
 Client8 Train => Local Epoch: 0 	Acc: 80.935 	Loss: 0.5533
 Client8 Test =>                   	Acc: 66.689 	Loss: 1.1501
 Client7 Train => Local Epoch: 0 	Acc: 80.161 	Loss: 0.5548
 Client

c:\Users\Arthur\anaconda3\envs\torch\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


 Client16 Test =>                   	Acc: 77.108 	Loss: 0.6446
------------------------------------------------
------ Federation process at Server-Side ------- 
------------------------------------------------
选择的客户端index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 11, 12, 13, 14, 15, 16, 17, 18, 19]
====================== SERVER V1==========================
 Train: Round  99, Avg Accuracy 77.966 | Avg Loss 0.694
 Test: Round  99, Avg Accuracy 62.575 | Avg Loss 1.374
idxs_users [ 8  7 17  0  9  3 12 13  6 19 18 11 10  4  1 14  5 15  2 16]
------------------------------------------------------------
------ Fed Server: Federation process at Client-Side -------
------------------------------------------------------------


c:\Users\Arthur\anaconda3\envs\torch\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


[8, 7, 17, 9, 3, 12, 13, 6, 18, 11, 10, 4, 1, 14, 5, 15, 2, 16]
fedserver选择的客户端index: [0, 1, 2, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
acc: 66.68911647796631
acc: 63.09940719604492
acc: 59.341325759887695
acc: 68.60856628417969
acc: 64.33189582824707
acc: 71.1274242401123
acc: 62.29121780395508
acc: 66.81707954406738
acc: 64.63496780395508
acc: 70.18453693389893
acc: 61.301185607910156
acc: 65.63846969604492
acc: 67.74649810791016
acc: 64.56088352203369
acc: 60.27747821807861
acc: 64.52047443389893
acc: 65.16702556610107
acc: 77.10802841186523
====================== Fed Server==========================
 Train: Round  99, Avg Accuracy 77.966 | Avg Loss 0.694
 Test: Round  99, Avg Accuracy 65.747 | Avg Loss 1.108
Epoch 99 finished in 00:02:00
Epoch 99 finished. Total time: 12091.26 seconds


In [15]:
print("Training and Evaluation completed!")

# ===============================================================================
# Save output data to .excel file (we use for comparision plots)
program = "SFLV1_label_flipping_random_detect_"
round_process = [i for i in range(1, len(acc_train_collect) + 1)]
df = DataFrame({'round': round_process, 'acc_train': acc_train_collect, 'acc_test': acc_test_collect,
                'loss_train': loss_train_collect, 'loss_test': loss_test_collect})
file_name = program  + str(poisoned_frac) + "_" + dataset_choice + ".xlsx"
krum_df = DataFrame({'round': round_process, 'acc_train': acc_train_collect, 'acc_test': krum_acc_test_collect,
                'loss_train': loss_train_collect, 'loss_test': krum_loss_test_collect})
krum_df.to_excel(file_name, sheet_name="v1_test", index=False)


Training and Evaluation completed!


In [16]:
acc_train_collect

[25.383042278289793,
 32.9638097333908,
 36.612591905593874,
 39.237591896057125,
 41.648092870712276,
 43.11948529720306,
 44.65705423355103,
 46.14234834194183,
 47.254825372695926,
 48.14947151184082,
 49.41693474769592,
 49.91865808486937,
 51.21886489868164,
 52.355238952636704,
 53.22277112960815,
 53.431066198349,
 54.289292297363275,
 55.331916389465334,
 55.71254594802856,
 56.220703134536734,
 57.164177370071414,
 57.50275732994081,
 58.3909696292877,
 58.87695308685303,
 59.302159910202036,
 60.144071674346925,
 60.548713178634635,
 60.604894285202036,
 61.38465068817139,
 61.70634188652038,
 62.092715959548954,
 62.32812494277955,
 63.16475177764894,
 63.280445728302006,
 63.87557439804078,
 63.916704883575434,
 64.28538599967956,
 64.58007809638977,
 64.78550084114075,
 65.41704958915709,
 65.83030785560608,
 66.20668651580809,
 66.26769295692442,
 66.386488904953,
 67.03527107238769,
 67.16440711021421,
 67.5899585533142,
 67.487476978302,
 68.1533776950836,
 68.413602895

In [17]:
import pandas as pd

# 创建一个名为df的DataFrame
df = pd.DataFrame({
    'name': ['Alice', 'Bob', 'Charlie'],
    'age': [25, 30, 35],
    'country': ['USA', 'Canada', 'Australia']
})